### Adrian Zaręba | 320672
# <span style="color:#458dc8"> Zagłębienie się w Sieci Kohonena (SOM)<span>

W tym dokumencie przyjrzymy się bliżej sieciom Kohonena, znane również jako Samoorganizujące się Mapy (SOM). Przejdziemy od teoretycznych podstaw, przez praktyczną implementację ich unikalnych właściwości, aż po zastosowania sieci w rozwiązywaniu realnych zadań (KOH2).

## Spis treści
1. [Opis Teoretyczny](#1-opis-teoretyczny) 
2. [Bazowa Implementacja](#2-bazowa-implementacja)
3. [Wgląd na finalną klasę](#3-finalna-klasa)
4. [Funkcje sąsiedztwa](#4-Funkcje-sąsiedztwa)
5. [Funkcje wygaszające](#5-wygaszanie)
6. [Warianty topologii](#7-warianty-topologii)
7. [Wizualizacje + Metody oceny](#8-wizualizacje)

## <span style="color:#458dc8"> 1. Opis Teoretyczny <span>

Sieci Kohonena są specjalnym typem sieci neuronowych, które skupiają się na nauce bez nadzoru i tworzeniu topologicznych map wejść, co pozwala na wizualizację dużych ilości danych w uproszczonej formie. SOM działają poprzez organizowanie siebie w przestrzeni dwuwymiarowej lub trójwymiarowej, co umożliwia łatwą analizę wzorców i klasteryzację danych (w naszym przypadku skupimy się na dwuwymiarowej).

### Model Matematyczny Neuronu w SOM

Podobnie jak inne sieci neuronowe, SOM operuje na zbiorze neuronów. Jednak w przeciwieństwie do tradycyjnych sieci, neurony w SOM są rozmieszczone na mapie topologicznej, gdzie każdy neuron jest bezpośrednio powiązany z określonymi sąsiadami, tworząc strukturę siatki.

$$ s = \arg\min_i \| x - w_i \|^2 $$

### Składniki Modelu

- $s$ to zwycięzca, czyli neuron, który najbardziej pasuje do aktualnie przetwarzanego wejścia $x$.
  
- $x$ to aktualne wejście do sieci, które jest porównywane z wagami neuronów w celu znalezienia zwycięzcy.

- $w_i$ to wagi i-tego neuronu na mapie. Wagi te są dostosowywane w procesie uczenia, co pozwala mapie samoorganizować się w sposób odzwierciedlający topologiczną strukturę danych wejściowych.

### <span style="color:lightblue"> Przydatna rada: <span>
W przeciwieństwie do sieci MLP, SOM mogą być nieco trudniejsze do 'wyobrażenia' (przynajmniej w moim wypadku). Warto jednak skupić uwagę na podstawy działania przez obliczenia matematyczne, co może zająć nieco więcej czasu, jednak na pewno warto.

### <span style="color:orange"> Uwaga: <span>
Podczas implementacji i eksploracji SOM, ważne jest, aby zrozumieć, że choć algorytm jest prosty w teorii, jego efektywność w praktyce może zależeć od wielu czynników, takich jak wybór parametrów uczących, inicjalizacja wag oraz sposób przetwarzania danych wejściowych. Dopasowanie tych elementów do specyficznych wymagań zadania jest kluczowe dla osiągnięcia optymalnych wyników.

## <span style="color:#458dc8"> 2. Bazowa Implementacja <span>
Spójrzmy na najprostszy szkielet naszej sieci KOH

In [ ]:
class KOH:
    def __init__(self, m, n, dim):
        self.m = m
        self.n = n
        self.dim = dim
        self.weights = np.random.rand(m, n, dim)

    def get_bmu(self, x):
        distances = np.linalg.norm(self.weights - x, axis=-1)
        return np.unravel_index(np.argmin(distances), (self.m, self.n))

### Opis Działania

#### <span style="color:lightblue"> Inicjacja Siatki Kohonena <span>

Przed rozpoczęciem działania sieci Kohonena musimy ją zainicjować. Metoda `__init__` przyjmuje trzy parametry:
- `m` oraz `n`, które określają wymiary siatki Kohonena (liczbę wierszy i kolumn odpowiednio).
- `dim`, który reprezentuje wymiarowość danych wejściowych.

Podczas inicjacji, dla każdej jednostki w siatce losowane są początkowe wagi o wymiarze `dim`. Te wagi stanowią punkty w przestrzeni, które sieć będzie adaptować podczas procesu uczenia.

#### <span style="color:lightblue"> Uczenie i Adaptacja Wagi <span>

Głównym celem sieci Kohonena jest samoorganizacja w celu zgrupowania podobnych danych wejściowych. W trakcie procesu uczenia, gdy podawany jest wektor wejściowy `x`, poszukujemy jednostki w siatce, której wagi najlepiej pasują do tego wektora. Ta jednostka nazywana jest Najlepszą Jednostką Dopasowującą (BMU).

#### <span style="color:lightblue"> Znajdowanie BMU <span>

Metoda `get_bmu` jest kluczowa w procesie uczenia. Przechodzi przez każdą jednostkę w siatce i oblicza odległość między wagami danej jednostki a wektorem wejściowym `x`. Następnie wybiera jednostkę o najmniejszej odległości, co odpowiada BMU. BMU wskazuje jednostkę w siatce, której wagi są najbliższe wektorowi wejściowemu.

#### <span style="color:lightblue"> Adaptacja Wag <span>

Po znalezieniu BMU, wagi tej jednostki oraz wag jej sąsiadów są dostosowywane, aby bardziej odpowiadały wektorowi wejściowemu. To dostosowanie zachodzi na zasadzie przyciągania wag w kierunku wektora wejściowego, co pozwala na lepsze dopasowanie do danych.

#### <span style="color:lightblue"> Iteracyjne Uczenie <span>

Proces znajdowania BMU i adaptacji wag jest iteracyjny. Dla każdego nowego wektora wejściowego sieć przeszukuje siatkę w poszukiwaniu BMU i dostosowuje wagi. Powtarzając ten proces dla wielu wektorów wejściowych, sieć samoorganizuje się, tworząc strukturę odpowiadającą rozkładowi danych w przestrzeni wejściowej.


#### <span style="color:orange"> Wyjaśnienie (skopiowane ze strony internetowej, pomocne) "Na Chłopski Rozum" <span>
##### Cytuję: "Wyobraź sobie sieć SOM jak mapę sklepów spożywczych w Twoim mieście. Każdy sklep ma swoje określone położenie na mapie. Kiedy masz listę zakupów (wektor danych wejściowych), szukasz najbliższego sklepu (BMU), aby zrobić zakupy. Gdy znajdziesz najbliższy sklep, idziesz tam i robisz zakupy (aktualizujesz wagi). Następnie, kiedy masz kolejną listę zakupów, powtarzasz ten proces, ucząc się, które sklepy są najlepiej dostosowane do Twoich potrzeb. W rezultacie, z czasem Twoje zakupy stają się coraz bardziej efektywne, ponieważ wiesz, które sklepy mają to, czego potrzebujesz, i są najbliżej Ciebie."

## <span style="color:#458dc8"> 3. Wgląd na finalną klasę <span>
Poniżej przedstawiona jest finalna klasa, na której bazował projekt. Posiada ona dużo funkcji w tym wizualizacyjne. W ramach teorii skupimy się jednak jedynie na najważniejszych fragmentach.
### KOH

In [5]:
class KOH:
    def __init__(self, m, n, dim, alpha=0.3, sigma=None, lambda_=100, neighborhood_func="gaussian", topology='rectangular'):
        """
        Inicjuje KOH z podanymi parametrami.
        Argumenty:
            m (int): Liczba wierszy w siatce KOH.
            n (int): Liczba kolumn w siatce KOH.
            dim (int): Wymiarowość danych wejściowych.
            alpha (float): Początkowa szybkość uczenia.
            sigma (float): Początkowa wielkość sąsiedztwa.
            lambda_ (float): Stała czasowa do zanikania.
            neighborhood_func (str): Typ funkcji sąsiedztwa ("gaussian" lub "mexican_hat").
            topology (str): Wybranie topologii sieci.
            weights (np.ndarray): Wagi siatki KOH.
            alpha_decay (function): Funkcja zanikająca szybkość uczenia.
            sigma_decay (float): Tempo zanikania sąsiedztwa.
        """
        self.m = m
        self.n = n
        self.dim = dim
        self.alpha = alpha
        self.sigma = sigma/100 if sigma is not None else max(m, n) / 2
        self.lambda_ = lambda_
        self.neighborhood_func = neighborhood_func
        self.topology = topology
        self.weights = np.random.rand(m, n, dim)
        self.alpha_decay = lambda t: np.exp(-t / self.lambda_)
        self.sigma_decay = 0.99


    def get_bmu(self, x):
        """
        Znajduje najlepszą jednostkę dopasowującą (BMU) dla podanego wektora wejściowego.
        Argumenty:
            x (np.ndarray): Wektor wejściowy.
        Zwraca:
            tuple: Współrzędne BMU w siatce KOH.
        """
        distances = np.linalg.norm(self.weights - x, axis=-1)
        return np.unravel_index(np.argmin(distances), (self.m, self.n))
    

    def hex_distance(self, a, b):
        """
        Oblicza odległość między dwoma punktami w zależności od topologii siatki.
        Argumenty:
            a (tuple): Współrzędne punktu startowego (ax, ay).
            b (tuple): Współrzędne punktu końcowego (bx, by).
        Zwraca:
            float: Obliczoną odległość między punktami w zależności od topologii.
        Wyrzuca:
            ValueError: Jeśli podana topologia nie jest obsługiwana (tylko 'rectangular' lub 'hexagonal').
        """
        ax, ay = a
        bx, by = b
        if self.topology == 'rectangular':
            return np.sqrt((ax - bx) ** 2 + (ay - by) ** 2)
        elif self.topology == 'hexagonal':
            dx = bx - ax
            dy = by - ay
            return max(abs(dx), abs(dy), abs(dx + dy))
        else:
            raise ValueError("Dostępne są tylko dwie topologie: rectangular | hexagonal")


    def get_neighborhood(self, bmu, sigma):
        """
        Tworzy macierz sąsiedztwa wokół podanego BMU (Best Matching Unit) z wykorzystaniem określonej topologii.
        Argumenty:
            bmu (tuple): Współrzędne BMU w siatce KOH.
            sigma (float): Bieżąca wielkość sąsiedztwa, określająca zakres, w jakim sąsiednie neurony są aktualizowane.
        Zwraca:
            np.ndarray: Macierz sąsiedztwa, gdzie wartości reprezentują wagę wpływu BMU na każdy neuron w siatce.
        """
        d = np.zeros((self.m, self.n))
        for i in range(self.m):
            for j in range(self.n):
                d[i, j] = self.hex_distance(bmu, (i, j))
        if self.neighborhood_func == "gaussian":
            return np.exp(-d / (2 * sigma ** 2))
        elif self.neighborhood_func == "mexican_hat":
            return (1 - d / (sigma ** 2)) * np.exp(-d / (2 * (sigma ** 2)))
        else:
            raise ValueError("Dostępne są tylko dwie funkcje: gaussian | mexican_hat")


    def update_weights(self, x, bmu, neighborhood):
        """
        Aktualizuje wagi siatki KOH.
        Argumenty:
            x (np.ndarray): Wektor wejściowy.
            bmu (tuple): Współrzędne BMU w siatce KOH.
            neighborhood (np.ndarray): Funkcja sąsiedztwa skupiona wokół BMU.
        """
        self.weights += self.alpha * neighborhood[..., None] * (x - self.weights)


    def train(self, data, num_iterations, labels=None, show_estimated_time=False):
        """
        Trenuje KOH na podanych danych.
        Argumenty:
            data (np.ndarray): Dane wejściowe.
            num_iterations (int): Liczba iteracji treningowych.
            show_estimated_time (bool): Opcja, czy wyświetlać estymowany czas wykonania.
        """
        try:
            iteration_time_sum = 0
            iteration_time_sq_sum = 0
            for t in range(num_iterations):
                iteration_start_time = time.time()
                for x in data:
                    bmu = self.get_bmu(x)
                    neighborhood = self.get_neighborhood(bmu, self.sigma)
                    self.update_weights(x, bmu, neighborhood)
                self.alpha = self.alpha_decay(t)
                self.sigma *= self.sigma_decay
                iteration_end_time = time.time()
                iteration_time = iteration_end_time - iteration_start_time
                iteration_time_sum += iteration_time
                iteration_time_sq_sum += iteration_time**2
                if show_estimated_time:
                    average_iteration_time = iteration_time_sum / (t + 1)
                    estimated_total_time = average_iteration_time * (num_iterations - t - 1)
                    print(f"Iteracja {t+1}/{num_iterations} | alpha: {self.alpha:.4f} | sigma: {self.sigma:.4f} | "
                          f"Estymowany czas pozostały: {estimated_total_time:.2f} sekund", end='\r')
        except KeyboardInterrupt:
            print("\nTraining interrupted. Finalizing...")
        except Exception as e:
            print(f"\nAn error occurred: {e}")
        finally:
            if labels is not None and len(labels) > 0:
                label_map = self.assign_labels(data, labels)
                accuracy = self.evaluate_accuracy(data, labels, label_map)
                print(f"\nAccuracy: {accuracy:.4f}%")
            else:
                print(f"\nFinal parameters | alpha: {self.alpha:.4f} | sigma: {self.sigma:.4f}")

    
    def assign_labels(self, data, labels):
        """
        Przypisuje etykiety do każdego neuronu na mapie Kohonena na podstawie dominującej etykiety najbliższych punktów danych.
        Argumenty:
            data (numpy.ndarray): Dane wejściowe.
            labels (numpy.ndarray): Odpowiadające etykiety dla danych wejściowych.
        Zwraca:
            numpy.ndarray: Mapa etykiet przypisanych do neuronów na mapie Kohonena.
        """
        label_map = np.zeros((self.m, self.n), dtype=int)
        for i in range(self.m):
            for j in range(self.n):
                distances = np.linalg.norm(data - self.weights[i, j], axis=1)
                closest_data_indices = np.argsort(distances)[:10]
                common_labels = labels[closest_data_indices]
                most_common = np.bincount(common_labels).argmax()
                label_map[i, j] = most_common
        return label_map


    def plot_kohonen_map(self, label_map):
        """
        Wyświetla mapę Kohonena z oznaczonymi klasami.
        Argumenty:
            label_map (numpy.ndarray): Mapa etykiet przypisanych do neuronów na mapie Kohonena.
        """
        fig, ax = plt.subplots()
        label_map = label_map.astype(int)
        cmap = plt.get_cmap('viridis', np.unique(label_map).max() + 1)
        mat = ax.matshow(label_map, cmap=cmap)
        cbar = plt.colorbar(mat, ticks=np.arange(np.min(label_map), np.max(label_map)+1))
        plt.title("Mapa Kohonena z etykietami klas")
        plt.show()


    def evaluate_accuracy(self, test_data, test_labels, label_map):
        """
        Ocenia dokładność klasyfikatora mapy Kohonena.
        Argumenty:
            test_data (numpy.ndarray): Dane testowe.
            test_labels (numpy.ndarray): Odpowiadające etykiety dla danych testowych.
            label_map (numpy.ndarray): Mapa etykiet przypisanych do neuronów na mapie Kohonena.
        Zwraca:
            float: Dokładność klasyfikatora mapy Kohonena.
        """
        predicted_labels = []
        for x in test_data:
            bmu = self.get_bmu(x)
            predicted_label = label_map[bmu]
            predicted_labels.append(predicted_label)
        accuracy = np.sum(predicted_labels == test_labels) / len(test_labels)
        return accuracy
    

    def calculate_silhouette_score(self, data, labels=None):
        """
        Oblicza Silhouette Score dla danych przetworzonych przez mapę Kohonena.
        Argumenty:
            data (np.ndarray): Dane wejściowe.
            labels (np.ndarray): Etykiety klastrów, jeśli już przypisane; w przeciwnym razie zostaną przypisane na podstawie BMU.
        Zwraca:
            float: Wartość Silhouette Score.
        """
        if labels is None:
            labels = np.array([self.get_bmu(x) for x in data])
            labels = labels[:, 0] * self.n + labels[:, 1]
        return silhouette_score(data, labels)


    def predict(self, data):
        """
        Przewiduje BMU dla każdego wektora wejściowego w danych.
        Argumenty:
            data (np.ndarray): Dane wejściowe.
        Zwraca:
            list: Lista BMU dla każdego wektora wejściowego.
        """
        return [self.get_bmu(x) for x in data]


    def plot_clusters(self, data, labels):
        """
        Rysuje wykres punktów danych pogrupowanych w klastry.
        Argumenty:
            data (np.ndarray): Dane wejściowe.
            labels (np.ndarray): Prawdziwe etykiety.
        Zwraca:
            dict: Słownik zawierający informacje o przypisaniu danych do klastrów.
        """
        bmus = [self.get_bmu(x) for x in data]
        label_dict = {tuple(bmu): labels[i] for i, bmu in enumerate(bmus)}
        cluster_labels = [label_dict[tuple(bmu)] for bmu in bmus]
        assignment_info = {i: f'Data row {i} assigned to cluster {cluster_labels[i]}' for i in range(len(data))}
        for info in assignment_info.values():
            print(info)
        if data.shape[1] == 2:
            plt.scatter(data[:, 0], data[:, 1], c=cluster_labels, cmap='viridis')
            plt.title('Clustering')
            plt.show()
        elif data.shape[1] == 3:
            fig = go.Figure(data=[go.Scatter3d(
                x=data[:, 0],
                y=data[:, 1],
                z=data[:, 2],
                mode='markers',
                marker=dict(size=5, color=cluster_labels, colorscale='Viridis', opacity=0.8)
            )])
            fig.update_layout(title='Clustering', scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), width=1100, height=550)
            fig.show()

## <span style="color:#458dc8"> 4. Funkcje sąsiedztwa <span>

W sieciach Kohonena, funkcje sąsiedztwa są kluczowymi elementami, które określają, jak bardzo neurony sąsiadujące z najlepiej dopasowaną jednostką (BMU) są modyfikowane podczas procesu uczenia. Te funkcje pomagają w regulowaniu wpływu BMU na swoje sąsiedztwo, co jest istotne dla prawidłowego rozmieszczenia i adaptacji wag neuronów na mapie cech. W klasie `KOH` mam zaimplementowane dwa typy funkcji sąsiedztwa: gaussowską i meksykański kapelusz.

#### <span style="color:lightblue"> Funkcja Gaussowska (Gaussian) <span>

Funkcja Gaussowska to standardowa funkcja używana w algorytmie Kohonena do modelowania wpływu BMU na jej sąsiednie neurony. Jest definiowana jako:

$$
N(d, \sigma) = \exp\left(-\frac{d^2}{2 \sigma^2}\right)
$$

gdzie $d$ jest odległością od BMU do danego neuronu, a $\sigma$ jest parametrem określającym wielkość sąsiedztwa, który z czasem maleje. Funkcja ta ma kształt dzwonu, co oznacza, że neurony bliżej BMU otrzymują silniejsze aktualizacje niż te dalsze.

#### <span style="color:lightblue"> Meksykański Kapelusz (Mexican Hat) <span>

Funkcja meksykański kapelusz, znana również jako funkcja Ricker wavelet, jest mniej standardowym wyborem, ale oferuje ciekawą alternatywę, która nie tylko wzmacnia neurony blisko BMU, ale również tłumi te znajdujące się w umiarkowanej odległości, przed ponownym zwiększeniem wpływu dla dalszych neuronów. Jest definiowana jako:

$$
N(d, \sigma) = (1 - \frac{d^2}{\sigma^2}) \exp\left(-\frac{d^2}{2 \sigma^2}\right)
$$

To połączenie funkcji liniowej i gaussowskiej prowadzi do profilu, który przypomina meksykański kapelusz, stąd nazwa (nie wiem dlaczego, ale bardzo mnie ciekawił ten fakt). Funkcja ta może sprzyjać tworzeniu bardziej zróżnicowanych klastrów, ponieważ wzmacnia oddziaływanie zarówno bliskich, jak i niektórych dalszych neuronów, przy jednoczesnym tłumieniu wpływu neuronów pośrednich.

#### <span style="color:lightblue"> Implementacja <span>

W klasie `KOH`, obie te funkcje są używane do generowania macierzy sąsiedztwa w metodzie `get_neighborhood`, która następnie jest wykorzystywana do aktualizacji wag w metodzie `update_weights`. Wybór funkcji sąsiedztwa (gaussowska czy meksykański kapelusz) wpływa na sposób, w jaki aktualizacje są stosowane do wag neuronów podczas treningu, co z kolei wpływa na ostateczny kształt mapy cech.

In [ ]:
def get_neighborhood(self, bmu, sigma):
    d = np.zeros((self.m, self.n))
    for i in range(self.m):
        for j in range(self.n):
            d[i, j] = self.hex_distance(bmu, (i, j))
    if self.neighborhood_func == "gaussian":
        return np.exp(-d / (2 * sigma ** 2))
    elif self.neighborhood_func == "mexican_hat":
        return (1 - d / (sigma ** 2)) * np.exp(-d / (2 * (sigma ** 2)))
    else:
        raise ValueError("Dostępne są tylko dwie funkcje: gaussian | mexican_hat")

## <span style="color:#458dc8"> 5. Funkcje wygaszające <span>
Funkcje wygaszające odgrywają kluczową rolę w algorytmach sieci Kohonena (SOM) poprzez kontrolowanie tempa uczenia się sieci w czasie. Zastosowanie tych funkcji pomaga w stabilizacji procesu uczenia, stopniowo zmniejszając wpływ każdego kolejnego wektora wejściowego, co pozwala na bardziej subtelne dostosowanie wag sieci w miarę jej "dojrzewania". W klasie `KOH` zaimplementowano dwie główne funkcje wygaszające dla szybkości uczenia (`alpha`) oraz wielkości sąsiedztwa (`sigma`).

#### <span style="color:lightblue"> Funkcja wygaszająca dla szybkości uczenia (`alpha_decay`)

Szybkość uczenia, `alpha`, jest jednym z najważniejszych parametrów w sieciach Kohonena, wpływającym na to, jak szybko sieć dostosowuje swoje wagi w odpowiedzi na każdy wektor wejściowy. Funkcja wygaszająca dla `alpha` jest zdefiniowana jako funkcja eksponencjalnego zaniku:

$$
\alpha(t) = \alpha_0 \cdot e^{-\frac{t}{\lambda}}
$$

gdzie:
- $\alpha_0$ to początkowa wartość szybkości uczenia,
- $t $ to numer iteracji,
- $ \lambda$ to stała czasowa, która kontroluje, jak szybko szybkość uczenia zanika w czasie.

#### <span style="color:lightblue"> Funkcja wygaszająca dla wielkości sąsiedztwa (`sigma_decay`)

Wielkość sąsiedztwa, `sigma`, określa zakres przestrzenny, w którym BMU wpływa na swoje sąsiednie neurony. Funkcja wygaszająca dla `sigma` w klasie `KOH` jest zaimplementowana jako prosta funkcja mnożnikowa:

$$
\sigma(t) = \sigma_0 \cdot d^{t}
$$

gdzie:
- $ \sigma_0 $ to początkowa wielkość sąsiedztwa,
- $d$ to współczynnik zaniku, zazwyczaj ustawiany na wartość poniżej 1 (np. 0.99), co oznacza, że `sigma` z każdą iteracją maleje o 1%.

#### <span style="color:lightblue"> Implementacja

Funkcje wygaszające są zaimplementowane w konstruktorze klasy `KOH` oraz używane w metodzie `train`, gdzie z każdą iteracją aktualizowane są wartości `alpha` i `sigma`:

In [ ]:
def __init__(self, m, n, dim, alpha=0.3, sigma=None, lambda_=100, ...):
    self.alpha = alpha
    self.sigma = sigma / 100 if sigma is not None else max(m, n) / 2
    self.lambda_ = lambda_
    self.alpha_decay = lambda t: np.exp(-t / self.lambda_)
    self.sigma_decay = 0.99

def train(self, data, num_iterations, ...):
    for t in range(num_iterations):
        self.alpha = self.alpha_decay(t)
        self.sigma *= self.sigma_decay
        ...

## <span style="color:#458dc8"> 6. Warianty topologii <span>
Topologia w sieciach Kohonena (SOM) odnosi się do sposobu rozmieszczenia neuronów oraz definicji sąsiedztwa między nimi. W naszej klasie `KOH` można wybrać jedną z dwóch topologii: prostokątną (rectangular) lub sześciokątną (hexagonal). Każda z nich wpływa na sposób interakcji neuronów, ich aktualizacji wag oraz ostateczną organizację mapy cech.

#### <span style="color:lightblue"> Topologia prostokątna (`rectangular`) <span>

W topologii prostokątnej, neurony są ułożone w regularnej siatce dwuwymiarowej. Sąsiedztwo jest zdefiniowane w sposób ortogonalny, co oznacza, że każdy neuron (oprócz tych na brzegach) ma czterech bezpośrednich sąsiadów (góra, dół, lewo, prawo).

##### <span style="color:lightblue"> Obliczanie odległości <span>

Odległość między neuronami jest mierzona standardowo, przy użyciu metryki euklidesowej:

$$
d(a, b) = \sqrt{(ax - bx)^2 + (ay - by)^2}
$$

gdzie $ (ax, ay) $ i $ (bx, by) $ to współrzędne dwóch różnych neuronów na siatce.

#### <span style="color:lightblue"> Topologia sześciokątna (`hexagonal`) <span>

Topologia sześciokątna jest bardziej złożona i pozwala na bliższe odwzorowanie naturalnych procesów grupowania, gdyż każdy neuron (oprócz tych na brzegach) ma sześciu sąsiadów. Taka konfiguracja zapewnia bardziej jednolite pokrycie przestrzeni i może prowadzić do bardziej homogenicznych map cech.

##### <span style="color:lightblue"> Obliczanie odległości <span>

Odległość między neuronami w topologii sześciokątnej jest zwykle mierzona przy użyciu metryki maksymalnej:

$$
d(a, b) = \max(|ax - bx|, |ay - by|, |ax + ay - bx - by|)
$$

Ten sposób obliczania odległości uwzględnia unikalny układ połączeń w siatce sześciokątnej, gdzie każdy neuron jest równo oddalony od swoich sześciu sąsiadów.

#### <span style="color:lightblue"> Znaczenie w praktyce <span>

Wybór topologii wpływa na sposób, w jaki dane wejściowe są klasyfikowane i reprezentowane na mapie Kohonena. Topologia sześciokątna zwykle oferuje lepsze pokrycie i może być bardziej efektywna w zachowywaniu topologicznych relacji w danych, podczas gdy prostokątna jest prostsza w implementacji i interpretacji.

#### <span style="color:lightblue">  Implementacja <span>

In [ ]:
def hex_distance(self, a, b):
    ax, ay = a
    bx, by = b
    if self.topology == 'rectangular':
        return np.sqrt((ax - bx) ** 2 + (ay - by) ** 2)
    elif self.topology == 'hexagonal':
        dx = bx - ax
        dy = by - ay
        return max(abs(dx), abs(dy), abs(dx + dy))
    else:
        raise ValueError("Dostępne są tylko dwie topologie: rectangular | hexagonal")

## <span style="color:#458dc8"> 7. Wizualizacje + Metody oceny <span>

Klasa `KOH` oferuje różne funkcje do wizualizacji i oceny wydajności mapy cech Kohonena. Poniżej znajduje si szczegółowy opis trzech kluczowych metod: `plot_kohonen_map`, `evaluate_accuracy` oraz `calculate_silhouette_score`.

#### <span style="color:orange"> Mapa Kohonena <span>

Metoda `plot_kohonen_map` służy do wizualizacji mapy Kohonena, przedstawiając neurony jako siatkę kolorowanych komórek, gdzie kolory odpowiadają różnym przypisanym klasom. Umożliwia to szybką ocenę sposobu, w jaki dane są klasyfikowane przez sieć.


#### <span style="color:orange"> Evaluate Accuracy <span>

Metoda `evaluate_accuracy` oblicza dokładność mapy Kohonena poprzez porównanie przewidywanych etykiet z rzeczywistymi etykietami testowymi. To narzędzie jest istotne do oceny jakości klasyfikacji wykonanej przez sieć.

Dokładność jest definiowana jako stosunek liczby poprawnie sklasyfikowanych próbek do ogólnej liczby próbek:

$$
\text{Accuracy} = \frac{\sum_{i=1}^{N} \mathbf{1}(y_i = \hat{y}_i)}{N}
$$

gdzie $ y_i $ to rzeczywista etykieta, a $ \hat{y}_i $ to przewidywana etykieta dla i-tej próbki.

#### <span style="color:orange"> Wskaźnik Silhouette <span>

Metoda `calculate_silhouette_score` oblicza wskaźnik Silhouette dla danych klasyfikowanych przez mapę Kohonena. Jest to miara, jak dobrze dane są grupowane w klastry.

Wskaźnik Silhouette jest średnią wartością wskaźnika Silhouette dla każdej próbki, gdzie:

$$
s = \frac{b - a}{\max(a, b)}
$$

- $ a $ jest średnią odległością między próbką a wszystkimi innymi punktami w tym samym klastrze,
- $ b $ jest minimalną średnią odległością od próbki do punktów w różnym klastrze.

Te trzy metody są kluczowe dla oceny i interpretacji wyników generowanych przez samoorganizującą się mapę Kohonena, dostarczając informacji o jakości klasyfikacji, wizualnej reprezentacji klastrów oraz optymalności grupowania danych.

# <span style="color:#458dc8"> Praktyczne zastosowanie znaleźć można w moim dodatkowym pliku KOH2 <span>
